This file contains a collection of solved SQL problems from Dataford, covering a range of topics from basic to advanced concepts. It's designed to enhance your understanding of SQL's powerful capabilities through practical, real-world examples.

In [ ]:
#SAME BIRTH MONTH
#Determine the count of employees within each department sharing the same birth month. The output should include the department, birth month, and the number of employees from that department who were born in that month.
SELECT DISTINCT
profession AS department,
birth_month,
COUNT(*) OVER (PARTITION BY profession, birth_month)
FROM employee_list
ORDER BY birth_month

In [ ]:
#HIGHEST AND LOWEST SALARIES
#You're tasked with finding employees with the highest and lowest salaries. Your output should feature the employee's ID, salary, and department.
#Additionally, include a column named 'salary_type' categorizing the results as either 'Highest Salary' or 'Lowest Salary' based on their respective salary rankings.
WITH CTE AS(
SELECT worker_id, salary, department,
RANK() OVER (PARTITION BY department, salary ORDER BY salary DESC) AS RankHighest,
RANK() OVER (PARTITION BY department, salary ORDER BY salary ASC) AS RankLowest
FROM worker
)
SELECT worker_id, salary, department,
CASE
WHEN RankHighest = 1 THEN 'Highest Salary'
WHEN RankLowest = 1 THEN 'Lowest Salary'
END AS salary_type
FROM CTE
WHERE RankHighest = 1 OR RankLowest = 1;

In [ ]:
#REVENUE ANALYSIS
#Identify companies whose revenue consistently increases every year without any dips. This means that if a company's revenue increases for several consecutive years but then experiences a dip in revenue, that company should not be included in the output.
WITH RevenueComparisons AS (
    SELECT
        company,
        year,
        revenue,
        LEAD(revenue) OVER (PARTITION BY company ORDER BY year) AS next_year_revenue
    FROM
        company_revenue
)

SELECT 
    DISTINCT company
FROM 
    RevenueComparisons
WHERE 
    revenue < next_year_revenue OR next_year_revenue IS NULL
GROUP BY 
    company
HAVING 
    COUNT(next_year_revenue) = COUNT(revenue);

In [ ]:
#USER ACTIVITY DURATION
#Your goal is to calculate the total active hours for each user. You should use the start and end times of user sessions, defined by the session state: '1' for session start and '0' for session end.
WITH SessionPairs AS (
    SELECT
        customer_id,
        state,
        timestamp,
        LEAD(timestamp) OVER (PARTITION BY customer_id ORDER BY timestamp) AS end_time
    FROM
        user_sessions
    WHERE
        state = 1
)

SELECT
    customer_id,
    SUM(EXTRACT(EPOCH FROM (end_time - timestamp)) / 3600.0) AS total_active_hours -- Convert seconds to hours
FROM
    SessionPairs
WHERE
    end_time IS NOT NULL
GROUP BY
    customer_id;

In [ ]:
#ROLLING AVERAGE TWEETS
#Write a query to calculate the 3-day rolling average of tweets for each user in the tweets table, and output the user ID, tweet date, and rolling averages rounded to 2 decimal places.
SELECT 
user_id, 
tweet_date,
ROUND(AVG(CAST(tweet_count) OVER (PARTITION BY user_id ORDER BY tweet_date
ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)),2) AS rolling_avg
FROM tweets
ORDER BY user_id, tweet_date;

In [ ]:
#MoM REVENUE CHANGE
#Write a SQL query to calculate the month-over-month percentage change in revenue.The output should include the year-month date (in YYYY-MM format) and percentage change, rounded to 2 decimal points.
SELECT
  TO_CHAR(purchase_date, 'YYYY-MM') AS year_month,
  ROUND(
    (SUM(revenue) - LAG(SUM(revenue), 1, 0) OVER (ORDER BY TO_CHAR(purchase_date, 'YYYY-MM'))) / 
    LAG(SUM(revenue), 1, NULL) OVER (ORDER BY TO_CHAR(purchase_date, 'YYYY-MM')) * 100,
    2
  ) AS percentage_change
FROM
  purchases
GROUP BY
  TO_CHAR(purchase_date, 'YYYY-MM')
ORDER BY
  year_month;

In [ ]:
#TRANSACTION RUNNING TOTAL
#Given a table of transactions with transaction dates and amounts, write a SQL query to calculate the running total for each day.For instance, if transactions on the first day total $20 and on the second day total $30, the output for the first day should be $20 and for the second day $50 (which is $20 from the first day plus $30 from the second).
SELECT transaction_date,
SUM(amount) OVER (ORDER BY transaction_date) AS running_total
FROM transactions
ORDER BY transaction_date;